This test saving random numbers and loading them via monkey-patching `torch.randn`

(currently not used)

In [1]:
import torch
import pickle

In [4]:
class RandomCacheSaver:
    def __init__(self, filename):
        self.cached_randoms = []
        self.filename = filename

    def start(self):
        self.orig_fn = torch.randn
        torch.rand = self.randn
    
    def randn(self, size):
        rands = torch.randn(size).cpu()
        self.cached_randoms.append((size, result))
        return result

    def stop(self):
        with open(self.filename, 'wb') as f: pickle.dump(self.cached_randoms, f)
        torch.rand = self.orig_fn

In [5]:
class RandomCacheLoader:
    def __init__(self, filename):
        with open(filename, 'rb') as f: self.cached_randoms = pickle.load(f)

    def start(self):
        self.orig_fn = torch.randn
        torch.rand = self.randn
    
    def randn(self, size):
        sz, rands = self.cached_randoms.pop(0)
        assert size==sz, f"Size of requested rands doesn't match size of cached rands: {size} != {sz}"
        return rands

    @property
    def n_remaining(self): return len(self.cached_randoms)
    
    def stop(self):
        if self.n_remaining>0: print('Info: Not all cached rands have been used')
        torch.rand = self.orig_fn

In [ ]:
# Usage for RandomCacheSaver
saver = RandomCacheSaver()
print(saver.rand(2))
saver.save('random_cache.pkl')

# Usage for RandomCacheLoader
loader = RandomCacheLoader('random_cache.pkl')
print(loader.rand(2))